In [ ]:
import MalardHelpers
import DataSetQuery
import datetime

#Sets the URL of the Malard ServiceGateway.
query = DataSetQuery.DataSetQuery('http://localhost:9000')

#Sets the output directory for all users of the server.
query.setEnvironment('test','/data/puma1/scratch/malard/export/')

#Setup the bounding box
minX=1300000
maxX=1400000
minY=0
maxY=100000
minT=datetime.datetime(2012,1,1,0,0)
maxT=datetime.datetime(2012,3,31,0,0)


In [ ]:
## TANDEMX
#Returns a file handle to a NetCDF that is a merge and filter of all the shards in a BoundingBox.
fileNameTandemX = query.getNetCdfFile('mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT)

print("File created: %s" % (fileNameTandemX))

#Converts the NetCDF into a DataFrame.
dfTandemX = MalardHelpers.getDataFrameFromNetCDF(fileNameTandemX)

print("TandemX Max Elevation %f" % (dfTandemX['elev'].max()))
print("TandemX Min Elevation %f" % (dfTandemX['elev'].min()))
print("TandemX Count Elevation %f" % (dfTandemX['elev'].count()))

In [ ]:
## SRTM
#Returns a file handle to a NetCDF that is a merge and filter of all the shards in a BoundingBox.
fileNameSrtm = query.getNetCdfFile('mtngla','srtm',minX,maxX,minY,maxY,minT,maxT)

print("File created: %s" % (fileNameSrtm))

#Converts the NetCDF into a DataFrame.
dfSrtm = MalardHelpers.getDataFrameFromNetCDF(fileNameSrtm)

print("SRTM Max Elevation %f" % (dfSrtm['elev'].max()))
print("SRTM Min Elevation %f" % (dfSrtm['elev'].min()))
print("SRTM Count Elevation %f" % (dfSrtm['elev'].count()))

In [ ]:
# Filter criteria

#a list of columns to see in the output (x,y,time will be added)
projections = ['elev','power','coh'] 
#a list of dictionaries that specify the filters
#only numeric fields are supported with the following operations gt(greater than), gte ( greater than equals), lt (less than) and lte (less than equals)
#filters are treated as and conditions
filters = [{'column':'power','op':'gt','threshold':10000},{'column':'coh','op':'gt','threshold':0.8}]

In [ ]:
# TANDEMX

#No filters
nofiltersTandemX = query.executeQuery( 'mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT,projections,[])
#Converts the NetCDF into a DataFrame.
dfNoFilterTandemX = MalardHelpers.getDataFrameFromNetCDF(nofiltersTandemX)

print( "TandemX Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( dfNoFilterTandemX['coh'].max(),  dfNoFilterTandemX['coh'].min(), dfNoFilterTandemX['power'].max(), dfNoFilterTandemX['power'].min(), dfNoFilterTandemX['power'].count() ) )  

#With filters
withfiltersTandemX = query.executeQuery( 'mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT,projections,filters)
#Converts the NetCDF into a DataFrame.
withFilterTandemX = MalardHelpers.getDataFrameFromNetCDF(withfiltersTandemX)

print( "TandemX Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( withFilterTandemX['coh'].max(),  withFilterTandemX['coh'].min(), withFilterTandemX['power'].max(), withFilterTandemX['power'].min(), withFilterTandemX['power'].count() ) )  
print( "TandemX Max Lon [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( withFilterTandemX['x'].max(),  withFilterTandemX['coh'].min(), withFilterTandemX['power'].max(), withFilterTandemX['power'].min(), withFilterTandemX['power'].count() ) )  
print( "TandemX Min Time [%f] Max Time [%f]" % ( withFilterTandemX['time'].min(),  withFilterTandemX['time'].max() ) )

In [ ]:
# SRTM

#No filters
nofiltersSrtm = query.executeQuery( 'mtngla','srtm',minX,maxX,minY,maxY,minT,maxT,projections,[])
#Converts the NetCDF into a DataFrame.
dfNoFilterSrtm = MalardHelpers.getDataFrameFromNetCDF(nofiltersSrtm)

print( "SRTM Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( dfNoFilterSrtm['coh'].max(),  dfNoFilterSrtm['coh'].min(), dfNoFilterSrtm['power'].max(), dfNoFilterSrtm['power'].min(), dfNoFilterSrtm['power'].count() ) )  

#With filters
withfiltersSrtm = query.executeQuery( 'mtngla','srtm',minX,maxX,minY,maxY,minT,maxT,projections,filters)
#Converts the NetCDF into a DataFrame.
withFilterSrtm = MalardHelpers.getDataFrameFromNetCDF(withfiltersSrtm)

print( "SRTM Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( withFilterSrtm['coh'].max(),  withFilterSrtm['coh'].min(), withFilterSrtm['power'].max(), withFilterSrtm['power'].min(), withFilterSrtm['power'].count() ) )  
print( "SRTM Max Lon [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( withFilterSrtm['x'].max(),  withFilterSrtm['coh'].min(), withFilterSrtm['power'].max(), withFilterSrtm['power'].min(), withFilterSrtm['power'].count() ) )  
print( "SRTM Min Time [%f] Max Time [%f]" % ( withFilterSrtm['time'].min(),  withFilterSrtm['time'].max() ) )

In [ ]:
from bokeh.plotting import figure, output_file
from bokeh.io import output_notebook, push_notebook, show
from bokeh.layouts import row, gridplot
from bokeh.models import ColumnDataSource, ColorBar, LinearColorMapper, LogColorMapper
from bokeh.palettes import Plasma8

# output to static HTML file
# output_file("TandemX_vs_SRTM_1300000_1400000_0_100000_1325376000000_1333148400000.html")

# output to notebook (below)
output_notebook()

# define data sources unfiltered
sourceTandemX = ColumnDataSource(dict(x=dfNoFilterTandemX['x'],y=dfNoFilterTandemX['y'], elev=dfNoFilterTandemX['elev']))
sourceSrtm = ColumnDataSource(dict(x=dfNoFilterSrtm['x'],y=dfNoFilterSrtm['y'], elev=dfNoFilterSrtm['elev']))
# data sources filtered
sourceTandemXF = ColumnDataSource(dict(x=withFilterTandemX['x'],y=withFilterTandemX['y'], elev=withFilterTandemX['elev']))
sourceSrtmF = ColumnDataSource(dict(x=withFilterSrtm['x'],y=withFilterSrtm['y'], elev=withFilterSrtm['elev']))

# Mapper
#mapperLin=LinearColorMapper(palette=Plasma8, low=withFilterTandemX['elev'].min(), high=withFilterTandemX['elev'].max())
mapperLog=LogColorMapper(palette=Plasma8, low=withFilterTandemX['elev'].min(), high=withFilterTandemX['elev'].max())
tools = "pan,wheel_zoom,zoom_in,zoom_out,box_zoom,box_select,hover,reset,"

# define tooltip attributes
tooltips = [
    ("(x,y)", "(@x{0}, @y{0})"),
    ("elevation", "@elev{0.00}"),
]

# create a TandemX plot
p1 = figure(plot_width=400, plot_height=350, title="TandemX", x_axis_label='x', y_axis_label='y', tools=tools, tooltips=tooltips)
p1.scatter(x='x', y='y', fill_alpha=0.6, line_color=None, fill_color={'field': 'elev', 'transform': mapperLog}, source=sourceTandemX)                     

# create a SRTM plot
p2 = figure(plot_width=400, plot_height=350, title="SRTM", x_axis_label='x', y_axis_label='y', tools=tools, tooltips=tooltips, x_range=p1.x_range, y_range=p1.y_range)
p2.scatter(x='x', y='y', fill_alpha=0.6, line_color=None, fill_color={'field': 'elev', 'transform': mapperLog}, source=sourceSrtm)

# create a TandemX plot filtered
p3 = figure(plot_width=400, plot_height=350, title="TandemX Filtered", x_axis_label='x', y_axis_label='y', tools=tools, tooltips=tooltips, x_range=p1.x_range, y_range=p1.y_range)
p3.scatter(x='x', y='y', fill_alpha=0.6, line_color=None, fill_color={'field': 'elev', 'transform': mapperLog}, source=sourceTandemXF)                     

# create a SRTM plot filtered
p4 = figure(plot_width=400, plot_height=350, title="SRTM Filtered", x_axis_label='x', y_axis_label='y', tools=tools, tooltips=tooltips, x_range=p1.x_range, y_range=p1.y_range)
p4.scatter(x='x', y='y', fill_alpha=0.6, line_color=None, fill_color={'field': 'elev', 'transform': mapperLog}, source=sourceSrtmF)


# Add colorbar
color_bar = ColorBar(color_mapper=mapperLog, width=8, location=(0,0))
p2.add_layout(color_bar, 'right')   

# show plots
show(gridplot([[p1, p2],[p3, p4]]))